# 特征分解在主成分分解 PCA 中的应用

2025-12
wawa

### 第一步：构造一个“有冗余信息”的二维数据

虽然是二维数据，但两个维度高度相关，直觉上“有效信息可能只有一维”。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)

n = 200
x1 = np.random.normal(0, 1, n) + 5      # 故意加偏移
x2 = 2 * x1 + np.random.normal(0, 0.5, n) + 10

X = np.column_stack((x1, x2))

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(X[:, 0], X[:, 1], alpha=0.6)
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Original Data Distribution")
plt.axis("equal")
plt.show()

- 点云明显沿着某个方向拉伸
- 两个特征不是“独立”的
- PCA 就是要找这条“最重要的方向”

### 第二步：数据中心化（减去均值）

PCA 只关心“变化”，不关心数据整体在什么位置。

In [ ]:
mean = X.mean(axis=0)
X_centered = X - mean

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], alpha=0.6)
plt.title("Before Centering")
plt.axis("equal")

plt.subplot(1, 2, 2)
plt.scatter(X_centered[:, 0], X_centered[:, 1], alpha=0.6)
plt.axhline(0)
plt.axvline(0)
plt.title("After Centering")
plt.axis("equal")

plt.show()

- 中心化后，数据以原点为中心
- 后面算协方差、特征值才有明确几何意义

我们可以看到，中心化之前数据整体偏离原点
- 协方差描述的其实是“围绕均值的波动”
- 如果不中心化，主成分方向会被“位置”干扰


### 第三步：协方差矩阵（数据结构的数学描述）

协方差矩阵描述的是“沿不同方向的变化程度”。

In [ ]:
cov = np.cov(X_centered, rowvar=False)
cov

$$
\begin{pmatrix}
1.0485 & 2.1337 \\
2.1337 & 4.5642
\end{pmatrix}
$$

先看对角线：每个特征自己的波动大小
- 第一个特征的方差是 1.05：说明它本身的波动程度不大，数据相对集中
- 第二个特征的方差是 4.56：明显比第一个大，说明它的取值范围更宽、变化更剧烈

两个维度本身“尺度就不一样”，第二维更“活跃”。

再看非对角线：两个特征之间的关系

- 协方差是 2.13，而且是正的
    - 两个特征相对于各自均值的“偏离方向”通常是同号的。    
- 数值还不小，这说明：
    - 当第一个特征变大时，第二个特征通常也会变大
    - 这里的“更大 / 更小”，都是相对于各自的均值，不是绝对值。
- 举个例子
    - 一个人：身高 160 cm（不算大）体重 80 kg（很大）
    - 另一个人：身高 190 cm（很大）体重 90 kg（也大）
    - 协方差关心的是：
        - 160 是否低于“平均身高”
        - 80 是否高于“平均体重”
        - 而不是 160 或 80 本身“大不大”。

两个特征之间存在强烈的正相关

相关系数：

$$
\rho = \frac{2.1337}{\sqrt{1.0485 \times 4.5642}} \approx 0.98
$$

这几乎是“接近线性关系”的级别。

> 这两个特征在很大程度上其实在重复表达同一件事。

从几何角度看这个协方差矩阵（这是 PCA 的直觉核心）

这个矩阵告诉我们三件事：

1. 数据云不是“圆形”，而是一个被拉伸的椭圆
2. 拉伸的主方向不是 x 轴或 y 轴
3. 真正的“变化方向”是某个斜着的方向

> 在二维里，一个“圆形分布”的协方差矩阵必须满足两个条件：
> - 两个方向的方差相等
> - 不存在相关性（协方差为 0）
>
> 在二维高斯数据中，有一个非常重要的事实：协方差矩阵的等概率线（或等方差线）一定是椭圆。
> - 特征值 → 椭圆两条轴的长度
> - 特征向量 → 椭圆的朝向
>
> 如果椭圆的主轴刚好对齐 x、y 轴，会发生什么？
> - x 和 y 方向是“相互独立的”
> - 协方差必须是 0
> - 对应的矩阵一定是对角矩阵：
> 
> 但我们的协方差 = 2.13 ≠ 0，这意味着：x 方向和 y 方向不是“自然的主方向”，是某个混合了 x 和 y 的斜方向。
> 如果我只沿着 x 或 y 看数据，都看不全；必须斜着看，才能看到最大的拉伸方向。
>
> 协方差矩阵的“快速判断口诀”
> - 协方差矩阵是对角的 → 主轴对齐坐标轴
> - 协方差矩阵有非零非对角项 → 主轴一定是斜的
> - 特征值相等 → 圆
> - 特征值不等 → 椭圆

这正是 PCA 要找的主成分方向。

因为存在很大的协方差，沿着原始坐标轴看方差并不高效，我们应该旋转坐标系，让一个轴对齐这个“共同变化方向”。

这个协方差矩阵表明：

数据在第二维度上波动更大，两个特征高度正相关，因此存在一个占据绝大部分方差的一维主方向，非常适合用 PCA 进行降维。

#### 3.1 协方差矩阵的特征值和每个维度的方差是什么关系？

一句话结论

> **协方差矩阵的特征值不是“某个原始维度的方差”，
> 而是“在某个方向（主成分方向）上的方差”。**

原始维度的方差在对角线上，特征值对应的是“旋转之后的新维度”的方差。

第一层：从定义上区分这两件事

给定一个二维随机向量 $(X_1, X_2)$，协方差矩阵是：

$$
\Sigma =
\begin{pmatrix}
\mathrm{Var}(X_1) & \mathrm{Cov}(X_1, X_2) \\
\mathrm{Cov}(X_1, X_2) & \mathrm{Var}(X_2)
\end{pmatrix}
$$

所以：

* 对角线元素
  → **原始特征轴上的方差**
* 非对角线元素
  → 特征之间的相关性

第二层：特征值在干什么（数学含义）

设 $v$ 是协方差矩阵的一个单位特征向量，对应特征值 $\lambda$：

$$
\Sigma v = \lambda v
$$

一个非常重要、但很多教材一笔带过的事实是：

$$
\mathrm{Var}(X \cdot v) = v^\top \Sigma v = \lambda
$$

这句话的含义是：

> **把数据投影到方向 $v$ 上，
> 投影后的方差正好等于对应的特征值。**

所以：

* 特征向量 → 方向
* 特征值 → 沿这个方向的数据方差

第三层：几何直觉（为什么要旋转）

想象你有一团“斜着拉长”的数据云：

* 沿 x 轴看 → 方差一般
* 沿 y 轴看 → 方差也一般
* 沿某条斜方向看 → 方差最大

协方差矩阵的特征分解做的事情是：

> 找到一组“最合适的坐标轴”，
> 让每个新轴上的方差彼此独立、且尽量集中。

在这组新坐标轴下：

* 坐标轴方向 = 特征向量
* 每个轴上的方差 = 特征值

#### 3.2 如果两个特征各自的方差都很大，但协方差很小，适合做PCA吗？

一句话结论

> **如果两个特征各自方差都很大，但协方差很小（接近 0），
> 那么一般“不适合”用 PCA 来做降维。**

更准确一点说：

> **PCA 不会带来明显的降维收益。**

为什么？从 PCA 想解决的问题说起

PCA 的目标不是“压缩数值大小”，而是：

> 用更少的维度，
> 表达原始数据中的“主要变化结构”。

这个“主要变化结构”出现的前提是：

* 不同特征之间存在**冗余**
* 即：它们在某种方向上“一起变化”

而“协方差很小”恰恰说明：

> 两个特征的变化基本是彼此独立的。

用协方差矩阵来直观看

如果两个特征：

* Var(X₁) 很大
* Var(X₂) 很大
* Cov(X₁, X₂) ≈ 0

协方差矩阵大致是：

$$
\Sigma \approx
\begin{pmatrix}
\sigma_1^2 & 0 \\
0 & \sigma_2^2
\end{pmatrix}
$$

这时会发生什么？

* 特征向量 ≈ 原始坐标轴
* 特征值 ≈ 原始各维度的方差

也就是说：

> PCA 找不到一个“比原始坐标轴更好”的方向。


从特征值角度看“是否值得降维”

判断 PCA 是否“有用”，一个非常实用的标准是：

> **最大的特征值是否远大于其他特征值。**

你的设定是：

* 两个特征方差都很大
* 协方差很小

通常意味着：

* 两个特征值都不小
* 也差不多大

例如：

$$
\lambda_1 \approx 10,\quad \lambda_2 \approx 8
$$

这时：

* PC1 只能解释 ~55% 的方差
* 丢掉 PC2 会损失大量信息

所以**不适合降到 1 维**。

总结

- PCA 适合用在“高方差 + 高相关”的特征组合上；
- 如果各维度彼此独立，即使方差很大，PCA 也不会带来有效的信息压缩。

#### 3.3 为什么正态分布数据的协方差矩阵的等方差线是个椭圆？

从**概率密度 → 二次型 → 几何**一步一步来解释。


> **二维正态分布的等概率（等方差）线一定是椭圆，
> 因为它们是一个正定二次型等于常数的集合。**


第一步：从二维正态分布的概率密度函数看

二维正态分布的概率密度是：

$$
p(x) = \frac{1}{2\pi |\Sigma|^{1/2}}
\exp\left(
-\frac{1}{2}
(x-\mu)^\top \Sigma^{-1} (x-\mu)
\right)
$$

关键点只有一个：

> 概率密度只取决于
> $(x-\mu)^\top \Sigma^{-1} (x-\mu)$

---

第二步：什么是“等概率 / 等方差线”

所谓“等概率线”，就是：$p(x) = \text{常数}$

这等价于：$(x-\mu)^\top \Sigma^{-1} (x-\mu) = c$

注意：
这是一个**二次型等于常数**。

---

第三步：二次型在二维空间里是什么形状？

在二维中，任何正定对称矩阵 $A$，满足$x^\top A x = c$

所描述的几何形状一定是： **椭圆**

原因是：

* $A$ 可以正交对角化
* 对角化后就是  $  \lambda_1 y_1^2 + \lambda_2 y_2^2 = c  $
* 再做变量替换，就变成标准椭圆方程

---

第四步：协方差矩阵为什么一定给出“正定二次型”

协方差矩阵 $\Sigma$ 有三个重要性质：

1. 对称
2. 半正定（实际数据中通常是正定）
3. 可正交特征分解

所以：

* $\Sigma^{-1}$ 也是对称正定的
* 对应的二次型一定是椭圆，而不可能是双曲线或抛物线

---

第五步：特征分解让“椭圆”完全可解释

对协方差矩阵做特征分解：$\Sigma = Q \Lambda Q^\top$

代回等概率方程：$(x-\mu)^\top Q \Lambda^{-1} Q^\top (x-\mu) = c$

令：$y = Q^\top (x-\mu)$

就得到：$\frac{y_1^2}{\lambda_1}* \frac{y_2^2}{\lambda_2} = c$

这已经是**标准椭圆方程**了。

于是可以清楚地对应：

* 特征向量 → 椭圆的主轴方向
* 特征值 → 主轴长度的平方（严格说是成反比关系）

---

第六步：为什么不是圆？

只有在一个特殊情况下才是圆： $\lambda_1 = \lambda_2$

也就是： 协方差矩阵 ∝ 单位矩阵， 各个方向的方差完全一样，这时椭圆退化成圆。

总结

- 正态分布的概率密度由一个二次型决定；
- 二维空间中，正定二次型的等值集合必然是椭圆；
- 协方差矩阵的特征分解，决定了椭圆的方向和拉伸程度。



### 第四步：特征值与特征向量（PCA 核心）

特征向量给方向，特征值给这个方向上有多少信息。

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(cov)

# 按特征值大小排序
idx = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]
eigenvalues, eigenvectors

特征值 $5.57,0.042$ 对应的特征向量：第一列：$\begin{pmatrix}-0.4267 \\-0.9044\end{pmatrix}$ 第二列：$\begin{pmatrix}-0.9044 \\0.4267\end{pmatrix}$

---

先看特征值：数据“往哪拉得最开”

特征值的含义是：**沿着对应特征向量方向的方差大小**。

这里两个数差距非常大，这意味着：

* 数据几乎所有的变化都集中在第一个方向上
* 垂直方向上的变化非常小

可以说：第一主成分解释了$ \frac{5.57}{5.57 + 0.042} \approx 99.3% $ 的方差。

几何直觉就是：数据云是一条被稍微“抖动”的细长椭圆，非常接近一条直线。

---

再看第一个特征向量：第一主成分（PC1）$(-0.4267, -0.9044)$

> 先说一个容易忽略但很重要的点：**正负号本身没有意义**。
> 
> $(-0.4267, -0.9044) \equiv (0.4267, 0.9044)$ 表示的是同一条方向，只是朝向相反。


这个向量在“说什么”

* 第二个分量的绝对值明显更大，方向大致是：x 向右一点，y 向上很多
- 这是那条“两个特征一起涨、一起跌”的斜方向。


第二个特征向量：被丢掉的方向（PC2）$(-0.9044, 0.4267)$

它和第一个特征向量：
- 完全正交，表示“变化最小”的方向
- 几何上就是：数据云的“厚度方向”。

这也是为什么：
- PCA 降维时直接丢掉它
- 损失的信息非常小

协方差矩阵的特征分解表明：

- 数据几乎全部沿着一个斜方向变化，该方向由第一个特征向量给出；
- 垂直于它的方向方差极小，因此可以安全地进行一维降维。


### 第五步：把“主成分方向”画出来

这是整个 PCA 演示最关键的一张图

In [ ]:
plt.figure(figsize=(6, 6))

# 数据点
plt.scatter(X_centered[:, 0], X_centered[:, 1],
            alpha=0.3, s=20)

# 原点
plt.scatter(0, 0, color="black", s=50)

pc1 = eigenvectors[:, 0]
pc2 = eigenvectors[:, 1]

# 用 sqrt(特征值) 缩放长度
len1 = np.sqrt(eigenvalues[0])
len2 = np.sqrt(eigenvalues[1])

scale = 2  # 视觉缩放

plt.quiver(0, 0,
           pc1[0]*len1, pc1[1]*len1,
           angles='xy', scale_units='xy', scale=1/scale,
           color='red', width=0.012)

plt.quiver(0, 0,
           pc2[0]*len2, pc2[1]*len2,
           angles='xy', scale_units='xy', scale=1/scale,
           color='blue', width=0.008)

# 标注
plt.text(pc1[0]*len1*scale*1.1,
         pc1[1]*len1*scale*1.1,
         "PC1 (large variance)",
         color='red', fontsize=11, weight='bold')

plt.text(pc2[0]*len2*scale*1.1,
         pc2[1]*len2*scale*1.1,
         "PC2 (small variance)",
         color='blue', fontsize=11)

plt.axhline(0, color='gray')
plt.axvline(0, color='gray')

plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("Principal Components with Variance Magnitude")
plt.axis("equal")
plt.show()


- 红色 PC1 非常长，说明沿这个方向数据被拉得很开；
- 蓝色 PC2 非常短，说明垂直方向上几乎没有变化。
- PCA 本质是在旋转坐标系

一个很重要但容易忽略的小提醒
- PC1 / PC2 的正负方向是不唯一的
- 如果你哪次看到箭头反过来，不是算错，而是数学上等价

在 PCA 里：
- 特征向量 → 椭圆主轴的方向
- √特征值 → 主轴的半径长度，不是特征值本身，而是平方根。

### 第六步：方差解释率（为什么可以降维）

如果第一主成分已经解释了大部分方差，就可以丢掉其他方向。

In [ ]:
explained_ratio = eigenvalues / eigenvalues.sum()
explained_ratio

In [ ]:
plt.figure(figsize=(4, 3))
plt.bar(["PC1", "PC2"], explained_ratio)
plt.ylabel("Explained Variance Ratio")
plt.title("Variance Explained by Each PC")
plt.show()


- PC1 占了绝大多数信息
- PC2 信息量很小
- 降维损失是“可控的”

### 第七步：投影到第一主成分（真正的降维）

降维不是“扔点”，而是“投影”。

In [ ]:
pc1 = eigenvectors[:, 0]

- eigenvectors 是一个 2×2 的矩阵，每一列是一个特征向量 `[:, 0]` 取的是第一列
- pc1 是第一主成分方向，一个单位向量，相当于原始二维平面里，我们选定的一条新的坐标轴。

In [ ]:
X_1d = X_centered @ pc1

- `X_centered`：形状是 `(n_samples, 2)`， `pc1`：形状是 `(2,)`
- `@` 是点积操作，每个数据点与 PC1 做点积，结果是每个二维样本 → 变成了一个标量。
- 点积在几何上意味着 = 投影长度
    - 这一步等价于：把每个二维点沿着 PC1 的方向投影到这条轴上，只保留在这条轴上的“坐标值”。
- 这一步就叫“降维”，因为：原来一个点需要两个数（x, y）描述，现在只需要一个数（在 PC1 上的位置）
- 降维过程，本质上就是：选择最大方差方向作为新坐标轴，然后把数据投影到这条轴上。

### 第八步：投影过程的几何可视化

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(X_centered[:, 0], X_centered[:, 1], alpha=0.3)

# 主成分方向
line = np.outer(X_1d, pc1)
plt.scatter(line[:, 0], line[:, 1], alpha=0.6)

plt.title("Projection onto First Principal Component")
plt.axis("equal")
plt.show()

- 所有点都“压”到一条线上
- 丢掉的是垂直方向的信息
- 保留的是变化最大的方向

### 第九步：一维结果展示（降维后的数据）

In [ ]:
plt.figure(figsize=(6, 2))
plt.scatter(X_1d, np.zeros_like(X_1d), alpha=0.6)
plt.yticks([])
plt.xlabel("PC1 Value")
plt.title("1D Data after PCA")
plt.show()


PCA 通过特征值和特征向量，把“相关的高维数据”
映射到“信息最集中的低维空间”。